In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
# from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix
from sklearn.linear_model import LogisticRegression

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder

In [27]:
X_train = pd.read_csv("../outputs/imputed_X_train_SMOTE.csv")
y_train = pd.read_csv("../outputs/imputed_y_train_SMOTE.csv")
X_test = pd.read_csv("../outputs/imputed_X_test_scaled.csv")
y_test = pd.read_csv("../outputs/imputed_y_test.csv")

# def map_df(data):

#     data["GenderCategory"] = data["GenderCategory"].map({1: "Female", 2: "Male"})
#     data["RaceCategory"] = data["RaceCategory"].map({1:"White", 2:"Black", 3:"Unknown"})
#     data["EthnicityCategory"] = data["EthnicityCategory"].map({1: "Non-Hispanic", 2:"Hispanic", 3: "Unknown"})
#     data["Region"] = data["Region"].map({1: "Northeast", 2: "Midwest", 3: "South", 4: "West", 5: "Unknown"})

#     return data

# X_train = map_df(X_train.copy(deep = True))
# X_test = map_df(X_test.copy(deep = True))

In [28]:
X_train.head()

,GenderCategory,RaceCategory,EthnicityCategory,Age,Region,Glipizide_Final,Glimepiride_Final,Glyburide_Final,Metformin_Final,Pioglitazone_Final,...,Eye_Disorder_Final,Atherosclerosis_Final,Alzheimer_Final,FootUlcer_Final,Abnormal_Glucose_Final,DMScreen_Final,A1C_Final,GlucoseTest_Final,DM_Drugs,AnyLab
0,Male,Black,Non-Hispanic,0.372434,Midwest,0,0,0,0,0,...,1,0,0,1,0,0,0,1,0,1
1,Female,White,Non-Hispanic,-0.512428,West,1,0,1,1,0,...,0,0,0,0,0,0,0,0,1,0
2,Female,White,Non-Hispanic,0.452876,Northeast,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Male,White,Non-Hispanic,0.452876,Northeast,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,Male,White,Non-Hispanic,-0.431986,Midwest,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [29]:
X_test.head()

,GenderCategory,RaceCategory,EthnicityCategory,Age,Region,Glipizide_Final,Glimepiride_Final,Glyburide_Final,Metformin_Final,Pioglitazone_Final,...,Eye_Disorder_Final,Atherosclerosis_Final,Alzheimer_Final,FootUlcer_Final,Abnormal_Glucose_Final,DMScreen_Final,A1C_Final,GlucoseTest_Final,DM_Drugs,AnyLab
0,Male,White,Non-Hispanic,0.050666,South,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,Male,White,Non-Hispanic,0.291992,West,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Male,White,Non-Hispanic,0.935529,Northeast,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Male,White,Non-Hispanic,-0.351544,South,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Male,White,Non-Hispanic,-0.271102,South,0,0,0,0,1,...,0,1,0,0,0,0,0,1,1,1


In [30]:
numerical_col = ["Age"]
categorical_col = list(X_train.columns)
categorical_col.remove("Age")
X_train[categorical_col] = X_train[categorical_col].astype('category')

In [31]:
categorical_features = X_train.columns[X_train.dtypes == 'category']
cat_col_numbers = [X_train.columns.get_loc(col) for col in X_train.select_dtypes(include="category")]
categorical_features

Index(['GenderCategory', 'RaceCategory', 'EthnicityCategory', 'Region',
       'Glipizide_Final', 'Glimepiride_Final', 'Glyburide_Final',
       'Metformin_Final', 'Pioglitazone_Final', 'Rosiglitazone_Final',
       'Beta_Blockers_Final', 'ACE_Inhibitors_Final', 'ARB_Final',
       'Diuretics_Final', 'PPI_Final', 'Levothyroxine_Final', 'CCB_Final',
       'Vasodilators_Final', 'Statins_Final', 'Anti_Platelets_Final',
       'Anti_Coagulants_Final', 'Steroids_Final', 'Heart_Disease_Final',
       'Hypothyroid_Final', 'Anemia_Final', 'Kidney_Disease_Final',
       'GERD_Final', 'Neuropathy_Final', 'Eye_Disorder_Final',
       'Atherosclerosis_Final', 'Alzheimer_Final', 'FootUlcer_Final',
       'Abnormal_Glucose_Final', 'DMScreen_Final', 'A1C_Final',
       'GlucoseTest_Final', 'DM_Drugs', 'AnyLab'],
      dtype='object')

In [32]:
lbl = LabelEncoder()
for col in categorical_col:
    X_train[col] = lbl.fit_transform(X_train[col])
    X_test[col] = lbl.transform(X_test[col])

## Feature selection

In [33]:
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import chi2

# def select_features(X_train, y_train, X_test):
#     fs = SelectKBest(score_func=chi2, k=10)
#     fs.fit(X_train.loc[:,categorical_col], y_train)
#     X_train_fs = fs.transform(X_train.loc[:,categorical_col])
#     X_test_fs = fs.transform(X_test.loc[:,categorical_col])

#     X_train_fs = pd.DataFrame(X_train_fs, columns = fs.f)




#     return X_train_fs, X_test_fs, fs


# a,b,c = select_features(X_train,y_train,X_test)
# for i in range(len(c.scores_)):
#  print('Feature %d: %f' % (i, c.scores_[i]))
# # plot the scores
# sns.barplot(x = [i for i in range(len(c.scores_))], y = c.scores_)

# Modelling

In [34]:
clf = LogisticRegression(penalty="l1", solver = "saga", max_iter=5000)
clf.fit(X_train,y_train.values.ravel())
clf_coeff = pd.DataFrame(clf.coef_, columns=X_train.columns)
clf_coeff = clf_coeff.T.reset_index()
clf_coeff[clf_coeff[0] == 0].index

Index([5, 18, 24, 25, 28, 32, 34, 36], dtype='int64')

In [35]:
X_train = X_train.drop(X_train.iloc[ :, [5, 18, 24, 25, 28, 32, 34, 36]].columns, axis = 1).reset_index(drop = True)
X_test = X_test.drop(X_test.iloc[:, [5, 18, 24, 25, 28, 32, 34, 36]].columns, axis = 1).reset_index(drop=True)

cat_col_numbers = [X_train.columns.get_loc(col) for col in X_train.select_dtypes(include="category")]

In [36]:
n_splits = 5

model_params = {
    "cat_1" : {
        "objective" : "CrossEntropy",
        "eval_metric" : 'AUC',
        "learning_rate" : 0.001, 
        "max_depth" : 7,
        # "random_state" : 56,
        "subsample" : 0.35,
        # 'early_stopping_rounds': 500,
        # 'ignored_features' : [4, 5, 18, 24, 25, 28, 32, 34, 36],
        "n_estimators": 10,
        "cat_features" : cat_col_numbers,
        "verbose" : 0        
    },


    "hgb_1" : {
        "learning_rate":0.015,
        "n_iter_no_change":100,
        "l2_regularization" : 0.02,
        "random_state" : 42,
        "categorical_features" : cat_col_numbers,
        "class_weight" : "balanced"
        
    },

    "lgbm_1" :
    {
        "objective" : "binary",
        "metric" : 'AUC',
        "learning_rate" : 0.018, 
        "max_depth" : 5,
        "random_state" : 56,
        "reg_alpha" : 0.01246,
        "reg_lambda": 0.023483,
        "subsample" : 0.35,
        "colsample_bytree" : 0.8,
        "verbose" : 0,
        "num_leaves" : 11,
        "categorical_features" : cat_col_numbers,
        "verbose" : -1,
        "class_weight" : "balanced"

    }
}

hgb_1 = HistGradientBoostingClassifier(**model_params["hgb_1"])
lgbm_1 = LGBMClassifier(**model_params["lgbm_1"])
# cat_1 = CatBoostClassifier(**model_params["cat_1"])

In [37]:
oofs_dict = {model + "_oof" : np.zeros(len(X_train)) for model in model_params.keys()}

In [38]:
def eval(y_true, y_pred, y_pred_proba):
    acc_score = accuracy_score(y_true, y_pred)
    conf_mat = confusion_matrix(y_true,y_pred)

    prec_score = precision_score(y_true, y_pred) 
    rec_score = recall_score(y_true, y_pred)
    F1_score = f1_score(y_true, y_pred)

    roc_auc = roc_auc_score(y_true, y_pred)
    roc_auc_predict_proba = roc_auc_score(y_true, y_pred_proba)

    print(f'Accuracy: {acc_score}')

    print("Precision_score :", prec_score)
    print("recall_score :", rec_score)
    print("f1_score:", F1_score)

    print(f'ROC AUC score: {roc_auc}')
    print(f'ROC AUC PROBA: {roc_auc_predict_proba}')

    print(conf_mat)

    print("==" * 25)

In [39]:
kf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
model_auc = {model + "_auc" : []  for model in model_params.keys()}


for num_fold, (train_index, val_index) in enumerate(kf.split(X_train,y_train.values)):
    
    print("="*20, "Fold : ", num_fold, "="*20)
    X_train_fold = X_train.iloc[train_index]
    X_val_fold = X_train.iloc[val_index]
    y_train_fold = y_train.iloc[train_index].values.ravel()
    y_val_fold = y_train.iloc[val_index].values.ravel()
    

    # cat_1.fit(X_train_fold,y_train_fold, eval_set=(X_val_fold,y_val_fold))
    # oofs_dict["cat_1_oof"][val_index] = cat_1.predict(X_val_fold)
    # y_pred = cat_1.predict(X_val_fold)
    # y_pred_proba = cat_1.predict_proba(X_val_fold)[:, 1]
    # # print('Fold', num_fold, 'CAT_1: ')
    # model_auc["cat_1_auc"].append(roc_auc_score(y_val_fold, y_pred))
    # eval(y_val_fold,y_pred, y_pred_proba)


    hgb_1.fit(X_train_fold, y_train_fold)
    oofs_dict["hgb_1_oof"][val_index] = hgb_1.predict(X_val_fold)
    y_pred = hgb_1.predict(X_val_fold)
    y_pred_proba = hgb_1.predict_proba(X_val_fold)[:, 1]
    model_auc["hgb_1_auc"].append(roc_auc_score(y_val_fold, y_pred))
    # print('Fold', num_fold, 'HGB_1: ')
    eval(y_val_fold,y_pred, y_pred_proba)
    
    
    lgbm_1.fit(X_train_fold, y_train_fold)
    oofs_dict["lgbm_1_oof"][val_index] = lgbm_1.predict(X_val_fold)
    y_pred = lgbm_1.predict(X_val_fold)
    y_pred_proba = lgbm_1.predict_proba(X_val_fold)[:, 1]
    # print('Fold', num_fold, 'LGBM_1: ')
    model_auc["lgbm_1_auc"].append(roc_auc_score(y_val_fold, y_pred))
    eval(y_val_fold,y_pred, y_pred_proba)


==================== Fold :  0 ====================
Accuracy: 0.8274111675126904
Precision_score : 0.8260869565217391
recall_score : 0.8715596330275229
f1_score: 0.8482142857142858
ROC AUC score: 0.8221434528773978
ROC AUC PROBA: 0.8751042535446205
[[68 20]
 [14 95]]
Accuracy: 0.8375634517766497
Precision_score : 0.7894736842105263
recall_score : 0.963302752293578
f1_score: 0.8677685950413224
ROC AUC score: 0.82256046705588
ROC AUC PROBA: 0.8934528773978316
[[ 60  28]
 [  4 105]]
==================== Fold :  1 ====================
Accuracy: 0.8316326530612245
Precision_score : 0.8220338983050848
recall_score : 0.8899082568807339
f1_score: 0.8546255506607929
ROC AUC score: 0.8242644732679532
ROC AUC PROBA: 0.8895919012970579
[[66 21]
 [12 97]]
Accuracy: 0.8061224489795918
Precision_score : 0.7709923664122137
recall_score : 0.926605504587156
f1_score: 0.8416666666666667
ROC AUC score: 0.7908889591901296
ROC AUC PROBA: 0.8835284192766002
[[ 57  30]
 [  8 101]]
==================== Fold : 

In [40]:
# sum(model_auc["cat_1_auc"])/5, sum(model_auc["hgb_1_auc"])/5, sum(model_auc["lgbm_1_auc"])/5


sum(model_auc["hgb_1_auc"])/5, sum(model_auc["lgbm_1_auc"])/5

(0.8387827979254743, 0.8218251799871542)

# HGB is better classifier than LGBM

### Training HGB on full train data and testing it before Hyperparameter tuning:

In [41]:
roc_list = []
kf = StratifiedKFold(n_splits=5, random_state=2023, shuffle=True)
for num_fold, (train_index, val_index) in enumerate(kf.split(X_train,y_train.values)):

    
    # print("="*20, "Fold : ", num_fold, "="*20)
    X_train_fold = X_train.iloc[train_index]
    X_val_fold = X_train.iloc[val_index]
    y_train_fold = y_train.iloc[train_index].values.ravel()
    y_val_fold = y_train.iloc[val_index].values.ravel()

    
    # hgb_1.fit(X_train_fold, y_train_fold)
    # oofs_dict["hgb_1_oof"][val_index] = hgb_1.predict(X_val_fold)
    # y_pred = hgb_1.predict(X_val_fold)
    # y_pred_proba = hgb_1.predict_proba(X_val_fold)[:, 1]
    # print('Fold', num_fold, 'HGB_1: ')
    # roc = roc_auc_score(y_val_fold,y_pred)
    # print("ROC: ", roc)
    # roc_list.append(roc)

    lgbm_1.fit(X_train_fold, y_train_fold)
    oofs_dict["hgb_1_oof"][val_index] = lgbm_1.predict(X_val_fold)
    y_pred = lgbm_1.predict(X_val_fold)
    y_pred_proba = lgbm_1.predict_proba(X_val_fold)[:, 1]
    print('Fold', num_fold, 'HGB_1: ')
    roc = roc_auc_score(y_val_fold,y_pred)
    print("ROC: ", roc)
    roc_list.append(roc)


print("Average ROC: ",sum(roc_list)/len(roc_list))
    # eval(y_val_fold,y_pred, y_pred_proba)

Fold 0 HGB_1: 
ROC:  0.8064011676396998
Fold 1 HGB_1: 
ROC:  0.8610144469049879
Fold 2 HGB_1: 
ROC:  0.8001159970473478
Fold 3 HGB_1: 
ROC:  0.8058631234841295
Fold 4 HGB_1: 
ROC:  0.8023832120636929
Average ROC:  0.8151555894279715


In [42]:
import optuna

def objective(trial,data=X_train,target=y_train.values.ravel()):
    train_x, val_x, train_y, val_y = train_test_split(data, target, test_size=0.33,random_state=42, stratify=target)
    params = {
        'l2_regularization': trial.suggest_float('l2_regularization',1e-2,10, log = True),
        'early_stopping': trial.suggest_categorical('early_stopping', [False, True]),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001,1, log = True),
        # 'max_iter': trial.suggest_categorical('max_iter', [750, 1000, 1250, 1500]),
        'max_depth': trial.suggest_int('max_depth', 2,5),
        'max_bins': trial.suggest_int('max_bins', 5,255),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 3,11),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 5,11),
        "categorical_features" : cat_col_numbers,
        "class_weight" : "balanced"
    }

    model = HistGradientBoostingClassifier(**params)
    model.fit(train_x, train_y)
    predictions = model.predict(val_x)
    auc = roc_auc_score(val_y, predictions)
    return auc

In [43]:
%%time
study = optuna.create_study(direction='maximize')
optuna.logging.set_verbosity(optuna.logging.ERROR)
study.optimize(objective, n_trials=999)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

Number of finished trials: 999
Best trial: {'l2_regularization': 0.11937774116167821, 'early_stopping': False, 'learning_rate': 0.9980880014489754, 'max_depth': 5, 'max_bins': 251, 'min_samples_leaf': 6, 'max_leaf_nodes': 10}
CPU times: user 20min 21s, sys: 25.1 s, total: 20min 46s
Wall time: 3min 31s


In [44]:
print(study.best_trial.value)

0.9104166666666665


In [45]:
hgb_1 = HistGradientBoostingClassifier(**study.best_trial.params)
hgb_1.fit(X_train,y_train.values.ravel())
y_pred = hgb_1.predict(X_test)
y_pred_proba = hgb_1.predict_proba(X_test)[:,1]
eval(y_test, y_pred, y_pred_proba)

Accuracy: 0.8121212121212121
Precision_score : 0.8843283582089553
recall_score : 0.8843283582089553
f1_score: 0.8843283582089553
ROC AUC score: 0.6921641791044777
ROC AUC PROBA: 0.7196677900818488
[[ 31  31]
 [ 31 237]]


# LGBM

In [46]:
def objective(trial,data=X_train,target=y_train.values.ravel()):
    train_x, val_x, train_y, val_y = train_test_split(data, target, test_size=0.33,random_state=42, stratify=target)
    params = {
        'reg_alpha': trial.suggest_float('reg_alpha',1e-3,10, log = True),
        'reg_lambda': trial.suggest_float('reg_lambda',1e-3,10, log = True),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001,1, log = True),
        'subsample': trial.suggest_float('sample', 0.01,1, log = True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01,1, log = True),
        # 'max_iter': trial.suggest_categorical('max_iter', [750, 1000, 1250, 1500]),
        'max_depth': trial.suggest_int('max_depth', 2,5),
        'max_bins': trial.suggest_int('max_bins', 5,255),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 3,11),
        'num_leaves': trial.suggest_int('num_leaves', 5,11),
        "categorical_features" : cat_col_numbers,
        "class_weight" : "balanced",
        "objective" : "binary",
        "metric" : 'AUC',
        "verbose" : -1


        # "objective" : "binary",
        # "metric" : 'AUC',
        # "learning_rate" : 0.018, 
        # "max_depth" : 5,
        # "reg_alpha" : 0.01246,
        # "reg_lambda": 0.023483,
        # "subsample" : 0.35,
        # "colsample_bytree" : 0.8,
        # "verbose" : 0,
        # "num_leaves" : 11,
        # "categorical_features" : cat_col_numbers,
        # "verbose" : -1,
        # "class_weight" : "balanced"
    }

    model = LGBMClassifier(**params)
    model.fit(train_x, train_y)
    predictions = model.predict(val_x)
    auc = roc_auc_score(val_y, predictions)
    print(auc)
    return auc

In [47]:
%%time
study = optuna.create_study(direction='maximize')
optuna.logging.set_verbosity(optuna.logging.ERROR)
study.optimize(objective, n_trials=100)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

0.8027777777777777
0.8486111111111111
0.8055555555555555
0.7958333333333333
0.8006944444444444
0.8055555555555555
0.79375
0.8062499999999999
0.8326388888888889
0.7944444444444445
0.8597222222222222
0.8631944444444444
0.8638888888888889
0.8465277777777778
0.7805555555555556
0.8659722222222221
0.8444444444444443
0.8486111111111111
0.9083333333333332
0.8666666666666666
0.8124999999999999
0.8854166666666666
0.8597222222222222
0.8493055555555556
0.8534722222222222
0.8493055555555555
0.8604166666666666
0.8159722222222222
0.8465277777777779
0.8743055555555556
0.8465277777777778
0.8722222222222222
0.8645833333333333
0.86875
0.8423611111111111
0.8506944444444444
0.8701388888888889
0.798611111111111
0.8284722222222223
0.8256944444444445
0.8541666666666665
0.8583333333333333
0.8701388888888889
0.8777777777777778
0.8618055555555555
0.8381944444444445
0.8430555555555557
0.8479166666666667
0.8527777777777777
0.8881944444444444
0.8951388888888889
0.8840277777777777
0.9152777777777779
0.88541666666666

In [48]:
lgbm_1 = LGBMClassifier(**study.best_trial.params)
lgbm_1.fit(X_train,y_train.values.ravel())
y_pred = lgbm_1.predict(X_test)
y_pred_proba = lgbm_1.predict_proba(X_test)[:,1]
eval(y_test, y_pred, y_pred_proba)

[LightGBM] [Warning] Unknown parameter: sample
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=4. Current value: min_data_in_leaf=4
[LightGBM] [Warning] Unknown parameter: sample
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=4. Current value: min_data_in_leaf=4
[LightGBM] [Info] Number of positive: 545, number of negative: 436
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000524 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 259
[LightGBM] [Info] Number of data points in the train set: 981, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.555556 -> initscore=0.223144
[LightGBM] [Info] Start training from score 0.223144
[LightGBM] [Warning] No further splits with positive gain, be

In [49]:
lgbm_1.feature_importances_

array([ 33,  23,   1, 317,  96,   3,  13,  10,   3,   2,  22,  31,  10,
        18,  19,  33,  23,  27,   5,   7,   7,  36,  19,  19,  13,  37,
        11,  14,  15,  15,  15], dtype=int32)